In [5]:
## try NER
import spacy

nlp = spacy.load("en_core_web_md")

In [7]:
import time

start_time = time.time()
doc = nlp("Located in a prime location on Le Duan Street, District 1, Diamond Department Store (Diamond Plaza) is a symbol of luxury shopping right in the heart of Saigon. It houses numerous high-end fashion, cosmetics, jewelry, and homeware brands from all over the world. Not just an ideal destination for shopping enthusiasts, Diamond Plaza also offers diverse culinary experiences and modern entertainment spaces, making it a perfect stop for a day of exploring Saigon with style and convenience.")
take_time = time.time() - start_time

In [8]:
print(f"Time taken: {take_time} seconds")
for token in doc:
    print(token.text, token.pos_, token.ent_type_)

Time taken: 0.05368375778198242 seconds
Located VERB 
in ADP 
a DET 
prime ADJ 
location NOUN 
on ADP 
Le PROPN FAC
Duan PROPN FAC
Street PROPN FAC
, PUNCT 
District PROPN DATE
1 NUM DATE
, PUNCT 
Diamond PROPN ORG
Department PROPN ORG
Store PROPN 
( PUNCT 
Diamond PROPN GPE
Plaza PROPN GPE
) PUNCT 
is AUX 
a DET 
symbol NOUN 
of ADP 
luxury NOUN 
shopping NOUN 
right ADV 
in ADP 
the DET 
heart NOUN 
of ADP 
Saigon PROPN GPE
. PUNCT 
It PRON 
houses VERB 
numerous ADJ 
high ADJ 
- PUNCT 
end NOUN 
fashion NOUN 
, PUNCT 
cosmetics NOUN 
, PUNCT 
jewelry NOUN 
, PUNCT 
and CCONJ 
homeware VERB 
brands NOUN 
from ADP 
all ADV 
over ADP 
the DET 
world NOUN 
. PUNCT 
Not PART 
just ADV 
an DET 
ideal ADJ 
destination NOUN 
for ADP 
shopping NOUN 
enthusiasts NOUN 
, PUNCT 
Diamond PROPN ORG
Plaza PROPN ORG
also ADV 
offers VERB 
diverse ADJ 
culinary ADJ 
experiences NOUN 
and CCONJ 
modern ADJ 
entertainment NOUN 
spaces NOUN 
, PUNCT 
making VERB 
it PRON 
a DET 
perfect ADJ 
stop NOUN 

In [9]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)
example = "My name is Wolfgang and I live in Berlin"

ner_results = nlp(example)
print(ner_results)

d:\HCMUS_ComputerScience\ComputationalThinking\TeamProject\reactProject\CLIP_Model\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\HCMUS_ComputerScience\ComputationalThinking\TeamProject\reactProject\CLIP_Model\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\PC\.cache\huggingface\hub\models--dslim--bert-base-NER. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you eit

[{'entity': 'B-PER', 'score': np.float32(0.9990139), 'index': 4, 'word': 'Wolfgang', 'start': 11, 'end': 19}, {'entity': 'B-LOC', 'score': np.float32(0.999645), 'index': 9, 'word': 'Berlin', 'start': 34, 'end': 40}]


In [4]:
import re

def heuristic_split(text):
    """
    Split text into sentences, avoiding splits inside numbers, abbreviations, etc.
    """
    # Bước 1: Replace dấu chấm trong các trường hợp đặc biệt bằng token tạm
    # Ví dụ: số kiểu 123.456, IP, version
    text = re.sub(r'(?<=\d)\.(?=\d)', '<DOT>', text)
    
    # Có thể thêm các abbreviations phổ biến: Mr., Dr., etc.
    abbreviations = ['Mr', 'Mrs', 'Dr', 'Ms', 'Prof']
    for abbr in abbreviations:
        text = re.sub(rf'\b{abbr}\.(?=\s)', f'{abbr}<DOT>', text)
    
    # Bước 2: Split theo dấu chấm / ! / ? thực sự
    sentences = re.split(r'(?<=[.!?])\s+', text)
    
    # Bước 3: Thay token <DOT> về lại dấu chấm
    sentences = [s.replace('<DOT>', '.') for s in sentences]
    
    return sentences

In [14]:
## read from ../DataCollector/result/places.db
## read rowid and desciption (is a string)
## split string by '.'
## create new db with (rowid, descrption in sentence)
## each sentence is a row

import sqlite3

conn = sqlite3.connect('../../DataCollector/result/places.db')
cursor = conn.cursor()

cursor.execute("SELECT description FROM places")
descs = cursor.fetchall()

list_add_to_new_db = []
for id, desc in enumerate(descs):
    desc = heuristic_split(desc[0])
    ## pop all '' string
    desc = [sentence for sentence in desc if sentence != '']
    for sentence in desc:
        list_add_to_new_db.append((id + 1, sentence))

conn.close()


In [15]:
print(list_add_to_new_db[:1])
reverse_list = list_add_to_new_db[::-1]
print(reverse_list[:100])

[(1, 'agataJapan at 19 Đặng Dung, District 1 is an ideal destination for those who love Japanese culture and products.')]
[(519, 'It features high-end brands, diverse cuisine, and modern cinemas, offering a world-class experience for all visitors and city residents.'), (519, 'Located in a prime location opposite Notre Dame Cathedral, Diamond Plaza is a bustling shopping and entertainment icon right in the heart of Saigon.'), (518, "It's the ideal spot for meeting, relaxing, and exploring the urban lifestyle."), (518, 'More than just shopping, Union Square is also a culinary paradise with a variety of luxurious restaurants and cafes, offering a modern, elegant space and a complete entertainment experience for all ages.'), (518, 'It brings together numerous international and domestic brands, from high-end fashion to sophisticated accessories.'), (518, 'Located right in the vibrant heart of Saigon, on the famous Đồng Khởi Street, Union Square Shopping Center is a high-class shopping and e

In [16]:
conn = sqlite3.connect('places_sentences.db')
cursor = conn.cursor()

cursor.execute("CREATE TABLE if not exists places_sentences (place_id INTEGER, sentence TEXT)")
for id_sentence in list_add_to_new_db:
    cursor.execute("INSERT INTO places_sentences (place_id, sentence) VALUES (?, ?)", id_sentence)

conn.commit()
conn.close()

print("Database created successfully")



Database created successfully


In [17]:

from sentence_transformers import SentenceTransformer
import torch
import numpy as np

# 1. Load model
model_name = "paraphrase-mpnet-base-v2"
model = SentenceTransformer(model_name)


In [18]:
## select all sentence from places_sentences sort by rowid
conn = sqlite3.connect('places_sentences.db')
cursor = conn.cursor()
cursor.execute("SELECT place_id, sentence FROM places_sentences")
result = cursor.fetchall()

conn.close()

index = [a[0] for a in result]
sentences = [a[1] for a in result]
print(index[:10])
print(sentences[:10])

[1, 1, 1, 1, 1, 2, 2, 2, 3, 3]
['agataJapan at 19 Đặng Dung, District 1 is an ideal destination for those who love Japanese culture and products.', 'Here, you will discover a sophisticated miniature world of the Land of Cherry Blossoms, from charming decor items, meticulous handicrafts, to practical products and fashion pieces imbued with distinct Japanese style.', 'Each product is carefully curated, ensuring quality and uniqueness.', 'agataJapan is not just a store, but also an elegant, peaceful experiential space amidst the vibrant heart of Saigon.', 'Come visit to immerse yourself and bring a touch of Japan home with you!', 'TOÀN UYÊN FOOD – The ideal destination for quality food!', 'Specializing in wholesale & retail of various delicious skewers, convenient frozen food, and fresh seafood.', "We ensure diverse and safe ingredients for all your family's culinary needs or your business, located at 3/12 De Tham Street, Co Giang Ward, District 1.", 'Welcome to My Vi Eatery at 35 Ky Con,

In [19]:
embeddings = model.encode(sentences, convert_to_tensor=True, batch_size=32, show_progress_bar=True)
embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
embeddings_np = embeddings.cpu().numpy()

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Batches: 100%|██████████| 44/44 [01:13<00:00,  1.67s/it]


In [20]:
import pickle

index = np.array(index, dtype=np.int32)
arrays = {
    'index': index,
    'embeddings': embeddings_np
}

with open('arrays.pkl', 'wb') as f:
    pickle.dump(arrays, f)



In [27]:
with open('arrays.pkl', 'rb') as f:
    arrays = pickle.load(f)

index = arrays['index']
embeddings = arrays['embeddings']

print(index[:10])
print(embeddings.shape)


[1 1 1 1 1 2 2 2 3 3]
(1834, 768)


In [30]:
sample_text = "Place to enjoy cultural banh mi in HCM"

query_embedding = model.encode([sample_text], convert_to_tensor=True)
query_embedding = torch.nn.functional.normalize(query_embedding, p=2, dim=1)
query_embedding_np = query_embedding.cpu().numpy()

# Tính toán cosine similarity giữa query embedding và tất cả embeddings
cosine_similarities = np.dot(embeddings, query_embedding_np.T)
print(cosine_similarities.shape)

(1834, 1)


In [33]:
print(np.argmax(cosine_similarities))

1191


In [50]:
## find out top k index with highest cosine similarity
k = 10
concate_id_and_similarity = np.concatenate((index.reshape(-1, 1), cosine_similarities), axis=1)
print(concate_id_and_similarity.shape)
top_k_indices = np.argsort(concate_id_and_similarity, axis=0)[-k:]
ans = top_k_indices[:, 1]
print(ans)


(1834, 2)
[ 452  607  410 1024  653  964  355 1234  912 1191]


In [51]:
print(cosine_similarities[ans])

[[0.6558473 ]
 [0.6569691 ]
 [0.65774906]
 [0.658208  ]
 [0.6693357 ]
 [0.6741901 ]
 [0.6888634 ]
 [0.71064216]
 [0.72482985]
 [0.7258788 ]]


In [52]:
print(top_k_indices[:10])

[[1824  452]
 [1825  607]
 [1826  410]
 [1827 1024]
 [1828  653]
 [1829  964]
 [1830  355]
 [1831 1234]
 [1832  912]
 [1833 1191]]


In [53]:
## query ans in places_sentences and print
ans_as_string = [str(a) for a in ans]
conn = sqlite3.connect('places_sentences.db')
cursor = conn.cursor()
cursor.execute("SELECT sentence FROM places_sentences WHERE rowid IN ({})".format(','.join(ans_as_string)))
result = cursor.fetchall()
conn.close()

print(result)
print(len(result))



[('',), ('',), ('',), ('Saigon vegetarian banh mi offers an irresistibly delicious flavor with a crispy baguette crust, a light and wholesome filling packed with fresh vegetables, tofu, and mushrooms, all richly seasoned',), ('',), ('',), (' You\'ll definitely be hopelessly "addicted"!',), (' An unforgettable culinary experience, enticing with every spoonful!',), ('"Bánh mì góc hẻm" – the name says it all about its rustic charm and the distinctive flavor of Saigon',), (' Savor a crispy, piping hot banh mi sandwich generously filled with fragrant pate, flavorful cold cuts, rich Vietnamese pork sausage, fresh herbs, and a touch of spicy chili',)]
10


In [54]:
result = [a[0] for a in result]
print('\n'.join(result))




Saigon vegetarian banh mi offers an irresistibly delicious flavor with a crispy baguette crust, a light and wholesome filling packed with fresh vegetables, tofu, and mushrooms, all richly seasoned


 You'll definitely be hopelessly "addicted"!
 An unforgettable culinary experience, enticing with every spoonful!
"Bánh mì góc hẻm" – the name says it all about its rustic charm and the distinctive flavor of Saigon
 Savor a crispy, piping hot banh mi sandwich generously filled with fragrant pate, flavorful cold cuts, rich Vietnamese pork sausage, fresh herbs, and a touch of spicy chili


In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification


d:\HCMUS_ComputerScience\ComputationalThinking\TeamProject\reactProject\CLIP_Model\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import spacy
nlp = spacy.load("xx_ent_wiki_sm") 
doc = nlp("Mr. John went to Starbucks Riverside.")
for ent in doc.ents:
    print(ent.text, ent.label_)

John PER
Starbucks Riverside ORG
